### 画最后一年年平均的经向热输送Heat Transport
#### Use NCL(6.6.2)

In [1]:
output                 = "/home/yyq/mls/dataprocess/photo/heat"
outputB                = "/home/yyq/mls/dataprocess/photo/heat_lat"
datapath               = "/data06/yyq/mls/licom2/addwater_new/exe/"
gridpath               = "/home/yyq/mls/dataprocess/nc/basin_new.nc"

In [2]:
fileslist              = systemfunc("ls "+ datapath+"MMEAN" +"*") 
f                      = addfiles(fileslist, "r") 
f_grid                 = addfile(gridpath, "r")

In [3]:
time_last              = dimsizes(fileslist) - 1

In [4]:
lev                    = f[0]->lev1
lat                    = f[0]->lat
vs_a                   = f[(time_last - 11):time_last]->vs
ts_a                   = f[(time_last - 11):time_last]->ts
indd                   = f_grid->indd

In [5]:
time_size              = dimsizes(vs_a(:,0,0,0)) - 1
lat_size               = dimsizes(vs_a(0,0,:,0)) - 1
lon_size               = dimsizes(vs_a(0,0,0,:)) - 1 - 2
lev_size               = dimsizes(lev) - 1

In [6]:
vs                     = vs_a(:,:,:,0:lon_size)
ts                     = ts_a(:,:,:,0:lon_size)
printVarSummary(vs)

Variable: vs
Type: float
Total Size: 29808000 bytes
            7452000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 12] x [lev | 30] x [lat | 115] x [lon | 180]
Coordinates: 
            time: [1057..1068]
            lev: [-5..-5243.597]
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 3
  _FillValue :	1e+35
  units :	m/s
  long_name :	meridional current

Constant

In [7]:
angle2radian= 4.0 * atan(1.0) / 180.0
r_earth = 6.371e6
dlon = 2
Cp = 4.18e3
rho0 = 1027

In [8]:
r = r_earth * cos (lat * angle2radian)
dx= r * (dlon * angle2radian)
dz = new((/lev_size/),typeof(lev))
do z = 1,lev_size
    dz(z-1) = ((lev(z) - lev(z-1))) * (-1)
end do 

In [9]:
dz_conform = new((/time_size+1,lev_size,lat_size+1,lon_size+1/),float)
dz_conform = conform(dz_conform, dz, (/1/))
dx_conform = new((/time_size+1,lev_size,lat_size+1,lon_size+1/),float)
dx_conform = conform(dx_conform, dx, (/2/))
printVarSummary(dx_conform)

Variable: dx_conform
Type: float
Total Size: 29808000 bytes
            7452000 values
Number of Dimensions: 4
Dimensions and sizes:	[12] x [30] x [115] x [180]
Coordinates: 
Number Of Attributes: 1
  _FillValue :	9.96921e+36

Integrate

In [10]:
heat = vs * ts * dz_conform * dx_conform * Cp * rho0                            
heat_integrate_lev =dim_sum_n(heat, 1)/(1e15)                                  
copy_VarMeta(vs(:,0,:,:), heat_integrate_lev)
heat_integrate_lev@long_name = "meridional heat transport"
heat_integrate_lev@units = "PW"
printVarSummary(heat_integrate_lev)

Variable: heat_integrate_lev
Type: float
Total Size: 993600 bytes
            248400 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 12] x [lat | 115] x [lon | 180]
Coordinates: 
            time: [1057..1068]
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 4
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  _FillValue :	1e+35

In [11]:
;all basin
heat_avg               = dim_avg_n_Wrap(heat_integrate_lev,0)                                     
heat_avg_lat           = dim_sum_n_Wrap(heat_avg,1)              
heat_avg_lat           = runave(heat_avg_lat, 9, 0)
;Pacfic
heat_avg_pacfic        = heat_avg
heat_avg_pacfic        = where(indd.eq.4, heat_avg, heat_avg@_FillValue)
heat_avg_lat_pacfic    = dim_sum_n_Wrap(heat_avg_pacfic,1)
heat_avg_lat_pacfic    = runave(heat_avg_lat_pacfic, 9, 0)
;Atlantic
heat_avg_atlantic      = heat_avg
heat_avg_atlantic      = where(indd.eq.2, heat_avg, heat_avg@_FillValue)
heat_avg_lat_atlantic  = dim_sum_n_Wrap(heat_avg_atlantic,1)
heat_avg_lat_atlantic  = runave(heat_avg_lat_atlantic, 9, 0)
;Indian
heat_avg_indian        = heat_avg
heat_avg_indian        = where(indd.eq.3, heat_avg, heat_avg@_FillValue)
heat_avg_lat_indian    = dim_sum_n_Wrap(heat_avg_indian,1)
heat_avg_lat_indian    = runave(heat_avg_lat_indian, 9, 0)

printVarSummary(heat_avg)
printVarSummary(heat_avg_lat)

Variable: heat_avg
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 5
  _FillValue :	1e+35
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  average_op_ncl :	dim_avg_n over dimension(s): timeVariable: heat_avg_lat
Type: float
Total Size: 460 bytes
            115 values
Number of Dimensions: 1
Dimensions and sizes:	[lat | 115]
Coordinates: 
            lat: [90..-78]
Number Of Attributes: 6
  _FillValue :	1e+35
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  average_op_ncl :	dim_avg_n over dimension(s): time
  sum_op_ncl :	dim_sum_n over dimension(s): lon

In [12]:
level                  = new((/51/),float)  
level                  = ispan(-25,25,1)
level                  = level / 10

Draw

In [13]:
wks      = gsn_open_wks("png", output)
plots    = new(1,"graphic")
res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@cnLevelSelectionMode      = "ExplicitLevels"
    res_0@cnLevels                  = level
    res_0@gsnLeftString             = "Meridional Heat Transport"
    res_0@gsnLeftStringFontHeightF  = 15
    res_0@gsnRightString            = "PW"
    res_0@gsnRightStringFontHeightF = 15
    res_0@mpCenterLonF              = 180
plots(0) = gsn_csm_contour_map(wks,heat_avg, res_0)

resP    = True
    resP@gsnPanelLabelBar           = True
    resP@lbLabelStride              = 5                 ;colorbar每5个色块一个标注
    resP@lbLabelAutoStride          = False             ;colorbar每条等值线均标注
    resP@lbBoxSeparatorLinesOn      = False             ;colorbar内部边框隐藏
    resP@lbBoxLinesOn               = False             ;colorbar外边框设置为隐藏
    resP@pmLabelBarHeightF          = 0.05              ;colorbar高度
    resP@pmLabelBarWidthF           = 0.9               ;colorbar宽度
    resP@pmLabelBarParallelPosF     = 0.025             ;colorbar水平偏移
    resP@lbBoxEndCapStyle           = "TriangleBothEnds"
gsn_panel(wks, plots, (/1,1/), resP)

.git   a.sh   ipynb  nc     ncl    photo  t

![heat](../photo/heat.png)

Draw lat_sum

In [14]:
y                      = new((/4,lat_size + 1/),float)
y(3,:)                 = heat_avg_lat
y(2,:)                 = heat_avg_lat_pacfic
y(1,:)                 = heat_avg_lat_atlantic
y(0,:)                 = heat_avg_lat_indian

In [15]:
wksB                              = gsn_open_wks("png", outputB)
plotsB                            = new(1,"graphic")
resB_0    = True
    resB_0@gsnDraw                = False
    resB_0@gsnFrame               = False           
    resB_0@vpHeightF              = 0.37
    resB_0@vpWidthF               = 0.6
    resB_0@tiXAxisString          = "Latitude" 
    resB_0@tiYAxisString          = "Heat Transport(PW)" 
    resB_0@gsnYRefLine            = 0
    resB_0@gsnYRefLineThicknessF  = 1.5
    resB_0@xyLineColors           = (/"purple","red","green","black"/)
    resB_0@xyDashPatterns         = (/0,0,0,0/)
    resB_0@xyLineThicknessF       = 3
    resB_0@trYMinF                = -2.5            
    resB_0@trYMaxF                = 2.5
    resB_0@trXMinF                = -35
    resB_0@trXMaxF                = 65

    resB_0@pmLegendDisplayMode    = "Always"
    resB_0@pmLegendOrthogonalPosF = -0.5
    resB_0@pmLegendParallelPosF   = 0.89
    resB_0@pmLegendWidthF         = 0.13
    resB_0@pmLegendHeightF        = 0.1
    resB_0@xyExplicitLabels       = (/"Indian","Atlantic","Pacific","Total"/)
plotsB(0)                          = gsn_csm_xy(wksB,lat,y, resB_0)

resBP                              = True
gsn_panel(wksB, plotsB, (/1,1/), resBP)

![heat_lat](../photo/heat_lat.png)